# TO USE GPU

USE GOOGLE COLLAB AND MOUNT YOUR DRIVE BY UNCOMMENTING THE FOLLOWING CODE (Ensure image folder is in your google drive)

Also, to enable the GPU on google colab, go to runtime tab and change runtime and add accelerator GPU

In [ ]:
## Mount google drive
#from google.colab import drive
#drive.mount("/content/gdrive")

# Convolution Neural Network - CNN Baseline

Convolution Neural Networks or CNN or Convnets are the current state of the art for most computer vision tasks.

This notebook will aply k-fold and use the concept of transfer learning.

Transfer learning:
- Train a CNN using 6 emotions dataset known as FER2013 dataset
- Strip out the top layers of that trained CNN architecture
- Freeze the weights of that architecture in hopes it learned high level features already
- Add back other layers that are trainable
- Train using our Smile dataset to learn specific features

In [24]:
# Import libraries and ignore warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
np.random.seed(2)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Plot architecture
import keras
import pydot
from keras.utils.vis_utils import model_to_dot

from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image

import os
from PIL import Image

# Data  
  
The first step is to import images and store their **pixel** values in a dataframe. This will allow us to create 10 fold by using the index of the rows.  
  
We will also **shuffle** the dataframe because the first 200 images are smilling and next 200 images are neutral.

In [18]:
# Read the annotations file that contains the label and the image file name
labels = pd.read_csv('./SMILE_Dataset/annotations.csv', header=None, names=['fname','label'])

# Shuffle data
labels = labels.sample(frac=1).reset_index()

# Use a list comprehension to loop over image file names and import one by one and store pixel values
x = np.array([image.img_to_array(image.load_img('./SMILE_Dataset/all/'+fname, target_size=(64, 64))) for fname in labels['fname']])

# Because the names are strings, the neural network only takes in numerical formats so we will one-hot encode the label
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels['label'])
y = integer_encoded

Now we have two variables.  
  
x: all of the values for our images  
y: all of the labels (0:1)
  
Now we have finished working with the data. Let's define an architecture for our CNN.

# Models

In [30]:
def pizza_model():
    
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Feed to a densily connected layer for prediction
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.Adam(lr=0.001),
                  metrics=['acc'])
    
    return model

# K-Fold

In [33]:
# All classification reports will be added here. When we are done we can average the f1 scores
reports = []

# Apply stratified K-fold ith 10 splits. Stratified means the same distribution of classes than the whole dataset
# In this case, 50-50
kf = StratifiedKFold(n_splits=10)

# Just for printing purposes
id = 1

for train_index, test_index in kf.split(x,y):
    print('Kfold iteration {}/10'.format(id))
    print('Total images: {} ---- Train images: {} ---- Test images: {}'.format(len(x),len(train_index),len(test_index)))

    id += 1 
    
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
        
    model = pizza_model()
    
    datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=10, # randomly rotate images in the range (degrees, 0 to 180)
                                 width_shift_range=0.1, # randomly shift images horizontally (fraction of total width)
                                 height_shift_range=0.1, 
                                 shear_range=0.1,
                                 zoom_range=0.1)   
    
    datagen.fit(X_train)

    # Secret sauce to get 3-5 % accuracy more
    # Adjust the learning rate over time. (Like we saw in class!)
    # The learning rate determines the size of the steps taken during the gradient descent process.
    
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
    
    # Used to prevent overfitting. 
    # es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    
    history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = 20), epochs = 20, 
                              validation_data = (X_test,y_test), steps_per_epoch=len(X_train) / 20,
                              callbacks=[learning_rate_reduction])

    
    y_pred = model.predict(X_test)
    y_pred = [np.round(p[0]) for p in y_pred]
    
    print(classification_report(y_test, y_pred))
    reports.append(classification_report(y_test, y_pred,output_dict=True))
    
#     if id == 3:
#         break

Kfold iteration 1/10
Total images: 400 ---- Train images: 360 ---- Test images: 40
Epoch 1/20
18/18 [==============================] - 4s 220ms/step - loss: 0.6950 - acc: 0.5306 - val_loss: 0.7912 - val_acc: 0.9000
Epoch 2/20
18/18 [==============================] - 3s 163ms/step - loss: 0.6342 - acc: 0.6806 - val_loss: 12.8143 - val_acc: 0.8250
Epoch 3/20
18/18 [==============================] - 4s 195ms/step - loss: 0.5684 - acc: 0.6778 - val_loss: 25.2818 - val_acc: 0.8250
Epoch 4/20
18/18 [==============================] - 4s 213ms/step - loss: 0.3904 - acc: 0.8417 - val_loss: 16.7153 - val_acc: 0.9500
Epoch 5/20
18/18 [==============================] - 4s 198ms/step - loss: 0.2551 - acc: 0.8889 - val_loss: 11.4652 - val_acc: 0.9250
Epoch 6/20
18/18 [==============================] - 3s 188ms/step - loss: 0.2135 - acc: 0.9278 - val_loss: 16.6521 - val_acc: 0.9250
Epoch 7/20
18/18 [==============================] - 3s 188ms/step - loss: 0.1658 - acc: 0.9361 - val_loss: 34.2032 - val

18/18 [==============================] - 3s 171ms/step - loss: 0.1098 - acc: 0.9556 - val_loss: 56.5493 - val_acc: 0.8750
Epoch 10/20
18/18 [==============================] - 3s 170ms/step - loss: 0.1154 - acc: 0.9639 - val_loss: 40.1345 - val_acc: 0.9000
Epoch 11/20
18/18 [==============================] - 3s 174ms/step - loss: 0.1034 - acc: 0.9528 - val_loss: 53.0449 - val_acc: 0.8500
Epoch 12/20
18/18 [==============================] - 3s 170ms/step - loss: 0.1063 - acc: 0.9694 - val_loss: 39.1880 - val_acc: 0.9000
Epoch 13/20
18/18 [==============================] - 3s 173ms/step - loss: 0.0830 - acc: 0.9667 - val_loss: 56.9871 - val_acc: 0.8500

Epoch 00013: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 14/20
18/18 [==============================] - 3s 172ms/step - loss: 0.0794 - acc: 0.9639 - val_loss: 46.3503 - val_acc: 0.9000
Epoch 15/20
18/18 [==============================] - 3s 172ms/step - loss: 0.0774 - acc: 0.9694 - val_loss: 43.0859 - val_acc: 

18/18 [==============================] - 3s 174ms/step - loss: 0.1014 - acc: 0.9639 - val_loss: 1.0355e-07 - val_acc: 1.0000
Epoch 18/20
18/18 [==============================] - 3s 172ms/step - loss: 0.1242 - acc: 0.9389 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 19/20
18/18 [==============================] - 3s 175ms/step - loss: 0.1094 - acc: 0.9528 - val_loss: 0.0000e+00 - val_acc: 1.0000

Epoch 00019: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
Epoch 20/20
18/18 [==============================] - 3s 171ms/step - loss: 0.1253 - acc: 0.9417 - val_loss: 0.0000e+00 - val_acc: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

Kfold iteration 6/10
Total images: 400 ---- Train images

18/18 [==============================] - 3s 182ms/step - loss: 0.3936 - acc: 0.8333 - val_loss: 4.6436 - val_acc: 0.9750
Epoch 4/20
18/18 [==============================] - 3s 178ms/step - loss: 0.2377 - acc: 0.9000 - val_loss: 9.4186 - val_acc: 0.9500
Epoch 5/20
18/18 [==============================] - 3s 181ms/step - loss: 0.1968 - acc: 0.9222 - val_loss: 6.5433 - val_acc: 0.9500
Epoch 6/20
18/18 [==============================] - 3s 178ms/step - loss: 0.1843 - acc: 0.9222 - val_loss: 3.9266 - val_acc: 0.9750

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/20
18/18 [==============================] - 3s 171ms/step - loss: 0.1394 - acc: 0.9556 - val_loss: 7.2081 - val_acc: 0.9750
Epoch 8/20
18/18 [==============================] - 4s 196ms/step - loss: 0.1288 - acc: 0.9528 - val_loss: 2.8567 - val_acc: 0.9750
Epoch 9/20
18/18 [==============================] - 4s 207ms/step - loss: 0.1348 - acc: 0.9500 - val_loss: 12.1633 - val_acc: 0.9250

Epoc

18/18 [==============================] - 3s 175ms/step - loss: 0.1341 - acc: 0.9611 - val_loss: 13.9925 - val_acc: 0.9750
Epoch 14/20
18/18 [==============================] - 3s 192ms/step - loss: 0.1358 - acc: 0.9444 - val_loss: 11.0922 - val_acc: 0.9750
Epoch 15/20
18/18 [==============================] - 3s 179ms/step - loss: 0.1053 - acc: 0.9611 - val_loss: 14.4425 - val_acc: 0.9750

Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 16/20
18/18 [==============================] - 3s 181ms/step - loss: 0.1030 - acc: 0.9778 - val_loss: 15.2298 - val_acc: 0.9500
Epoch 17/20
18/18 [==============================] - 3s 182ms/step - loss: 0.0971 - acc: 0.9639 - val_loss: 17.7755 - val_acc: 0.9750
Epoch 18/20
18/18 [==============================] - 3s 179ms/step - loss: 0.1194 - acc: 0.9583 - val_loss: 13.5198 - val_acc: 0.9750

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 19/20
18/18 [===========================

In [34]:
# We loop over all reports (1 per fold) and then compute the average of all weighted f1 scores
final_f1_score = np.mean([rep['weighted avg']['f1-score'] for rep in reports])

print('Final F1-Score is: {}%'.format(np.round(final_f1_score*100,2)))

Final F1-Score is: 96.49%
